# Lorenz system: manifold plots

In [ ]:
%matplotlib osx

import sys
import numpy as np
import matplotlib
import torch
import logging
import pickle
from tqdm import tqdm
from scipy.spatial import cKDTree

from matplotlib import pyplot as plt, cm
from matplotlib.collections import PolyCollection
from matplotlib.tri import Triangulation
from matplotlib.colors import LightSource
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

sys.path.append("../../")
from experiments.datasets import LorenzSimulator
from experiments.architectures.vector_transforms import create_vector_transform
from manifold_flow.flows import ManifoldFlow
import plot_settings as ps

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-30.30s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.DEBUG,
)
for key in logging.Logger.manager.loggerDict:
    if "experiments" not in key and "manifold_flow" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [ ]:
# ps.setup()

## Set up simulator

In [ ]:
sim = LorenzSimulator()


## Load flows

In [ ]:
def load_model(filename):
    outer_transform = create_vector_transform(
        3,
        5,
        linear_transform_type="permutation",
        base_transform_type="rq-coupling",
        context_features=None,
        dropout_probability=0.,
        tail_bound=3.,
        num_bins=5,
    )
    inner_transform = create_vector_transform(
        2,
        5,
        linear_transform_type="permutation",
        base_transform_type="rq-coupling",
        context_features=sim.parameter_dim(),
        dropout_probability=0.,
        tail_bound=3.,
        num_bins=5,
    )

    model = ManifoldFlow(
        data_dim=3,
        latent_dim=2,
        outer_transform=outer_transform,
        inner_transform=inner_transform,
        apply_context_to_outer=False,
    )
    
    model.load_state_dict(
        torch.load("../data/models/{}.pt".format(filename), map_location=torch.device("cpu"))
    )
    _ = model.eval()
    
    return model

In [ ]:
mf = load_model("mf_2_lorenz_april")


## Find points for evaluation: start with latent grid, triangulate, add points randomly where necessary, remove duplicates

In [ ]:
def make_grid(boundary, res):
    grid_each = np.linspace(-boundary, boundary, res)
    x, y = np.meshgrid(grid_each, grid_each)
    xy = np.vstack((x.flatten(), y.flatten())).T
    return xy


def decode(zs, batchsize = 10000):
    xs = []
    n_batches = (len(zs) - 1) // batchsize + 1
    for i in range(n_batches):
        print("Batch {} / {}".format(i + 1, n_batches))
        z_batch = zs[i*batchsize:(i+1)*batchsize]
        z_ = torch.tensor(z_batch, dtype=torch.float)
        x_ = mf.decode(u=z_)
        xs += list(x_.detach().numpy().flatten())

    xs = np.array(xs).reshape((-1, 3))
    return xs


def encode(xs, batchsize = 10000):
    zs = []
    n_batches = (len(xs) - 1) // batchsize + 1
    for i in range(n_batches):
        print("Batch {} / {}".format(i + 1, n_batches))
        x_batch = xs[i*batchsize:(i+1)*batchsize]
        x_ = torch.tensor(x_batch, dtype=torch.float)
        z_ = mf.encode(x_)
        zs += list(z_.detach().numpy().flatten())

    zs = np.array(zs).reshape((-1, 2))
    return zs


def triangle_circumference(a, b, c):
    ab = np.linalg.norm(a - b)
    bc = np.linalg.norm(b - c)
    ca = np.linalg.norm(c - a)
    circ = (ab + bc + ca)
    return circ


def triangle_area(a, b, c):
    ab = np.linalg.norm(a - b)
    bc = np.linalg.norm(b - c)
    ca = np.linalg.norm(c - a)
    s = (ab + bc + ca) / 2.
    return (s * (s-ab) * (s-bc) * (s-ca))**0.5
    
    
def uniform_points_in_triangle(a, b, c, n=1):
    p = np.random.uniform(size=n)[:,np.newaxis]
    q = np.random.uniform(size=n)[:,np.newaxis]
    p_ = np.where(p + q > 1., 1. - p, p)
    q_ = np.where(p + q > 1., 1. - q, q)
    points = a[np.newaxis, :] + (b - a)[np.newaxis, :] * p_ + (c - a)[np.newaxis, :] * q_
    
    return points


In [17]:
def iterate(xs, zs, goal_area = 0.01, goal_circ=0.2, insert_max=10, max_distance=0.05):
    print("Beginning iteration")
    n_in = xs.shape[0]
    z_outs = np.copy(zs)
    
    # Step one: inserting more points where distances are large
    print("Triangulating...")
    triangulation = Triangulation(zs[:,0], zs[:,1])

    print("Inserting points in latent space...")
    for t in triangulation.triangles:
        area = triangle_area(xs[t[0]], xs[t[1]], xs[t[2]])
        circ = triangle_circumference(xs[t[0]], xs[t[1]], xs[t[2]])
        try:
            n_additional = int(round(np.clip(
                0.5 * (area / goal_area + circ / goal_circ),
                0., insert_max
            ), 0))
        except:
            n_additional = insert_max // 2  # Seems like a safe choice

        if n_additional > 0:
            z_additional = uniform_points_in_triangle(
                zs[t[0]],
                zs[t[1]],
                zs[t[2]],
                n=n_additional
            )
            z_outs = np.concatenate((z_outs, z_additional), axis=0)
            
    print("Decoding...")
    x_outs = decode(z_outs)
    n_mid = len(x_outs)

    # Step two: removing close neighbors in x space
    print("Removing close neighbors in data space...")
    tree = cKDTree(x_mf)
    neighbors = tree.query_pairs(r=max_distance)
    duplicates = [n[1] for n in neighbors]

    x_outs = np.array([x for i, x in enumerate(x_outs) if i not in duplicates])
    z_outs = np.array([z for i, z in enumerate(z_outs) if i not in duplicates])
    n_out = len(x_outs)
    
    print(f"Done! {n_in} -> {n_mid} -> {n_out}")

    return x_outs, z_outs

In [18]:
z_mf = make_grid(2.9, 100)
x_mf = decode(z_mf)


Batch 1 / 1


In [19]:
for _ in range(2):
    x_mf, z_mf = iterate(x_mf, z_mf)


Beginning iteration
Triangulating...
Inserting points in latent space...
Decoding...
Batch 1 / 5
Batch 2 / 5
Batch 3 / 5
Batch 4 / 5
Batch 5 / 5
Removing close neighbors in data space...
Done! 10000 -> 41631 -> 35244
Beginning iteration
Triangulating...
Inserting points in latent space...
Decoding...
Batch 1 / 14
Batch 2 / 14
Batch 3 / 14
Batch 4 / 14
Batch 5 / 14
Batch 6 / 14
Batch 7 / 14
Batch 8 / 14
Batch 9 / 14
Batch 10 / 14
Batch 11 / 14
Batch 12 / 14
Batch 13 / 14
Batch 14 / 14
Removing close neighbors in data space...
Done! 35244 -> 132713 -> 103626
Beginning iteration
Triangulating...


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars


Inserting points in latent space...
Decoding...
Batch 1 / 38
Batch 2 / 38
Batch 3 / 38
Batch 4 / 38
Batch 5 / 38
Batch 6 / 38
Batch 7 / 38
Batch 8 / 38
Batch 9 / 38
Batch 10 / 38
Batch 11 / 38
Batch 12 / 38
Batch 13 / 38
Batch 14 / 38
Batch 15 / 38
Batch 16 / 38
Batch 17 / 38
Batch 18 / 38
Batch 19 / 38
Batch 20 / 38
Batch 21 / 38
Batch 22 / 38
Batch 23 / 38
Batch 24 / 38
Batch 25 / 38
Batch 26 / 38
Batch 27 / 38
Batch 28 / 38
Batch 29 / 38
Batch 30 / 38
Batch 31 / 38
Batch 32 / 38
Batch 33 / 38
Batch 34 / 38
Batch 35 / 38
Batch 36 / 38
Batch 37 / 38
Batch 38 / 38
Removing close neighbors in data space...


KeyboardInterrupt: 

## Evaluate log likelihood

In [ ]:
batchsize = 600
n_batches = (len(z_mf) - 1) // batchsize + 1

logps = []

for i in range(n_batches):
    print("Batch {} / {}".format(i + 1, n_batches))
    x_batch = x_mf[i*batchsize:(i+1)*batchsize]
    x_batch_ = torch.tensor(x_batch, dtype=torch.float)
    _, logp_, _ = mf(x_batch_, mode="mf")
    logps.append(logp_.detach().numpy().flatten())

logp_mf = np.hstack(logps)
logp_mf.shape


Batch 1 / 173
Batch 2 / 173
Batch 3 / 173
Batch 4 / 173
Batch 5 / 173
Batch 6 / 173
Batch 7 / 173
Batch 8 / 173
Batch 9 / 173
Batch 10 / 173
Batch 11 / 173
Batch 12 / 173
Batch 13 / 173
Batch 14 / 173
Batch 15 / 173
Batch 16 / 173
Batch 17 / 173
Batch 18 / 173


In [ ]:
x_traj = sim.trajectory()
z_traj = encode(x_traj, batchsize=10000)
x_traj.shape, z_traj.shape


## Save / load results

In [ ]:
pickle.dump((x_mf, z_mf, logp_mf, x_traj, z_traj), open( "../data/results/lorenz2.pickle", "wb"))


In [ ]:
# x_mf, z_mf, logp_mf, x_traj, z_traj = pickle.load(open( "../data/results/lorenz.pickle", "rb"))


## Triangulation

In [ ]:
circ_max = 0.8
triangulation = Triangulation(z_mf[:,0], z_mf[:,1])

x_tri = []
logp_tri = []
z_tri = []
minlogp_tri = []

for t in triangulation.triangles:
    circ = triangle_circumference(x_mf[t[0]], x_mf[t[1]], x_mf[t[2]])
    if circ_max is not None and circ > circ_max:
        continue
        
    z_tri.append(
        [
            [z_mf[t[0], 0], z_mf[t[0], 1]],
            [z_mf[t[1], 0], z_mf[t[1], 1]],
            [z_mf[t[2], 0], z_mf[t[2], 1]]
        ]
    )
    x_tri.append(
        [
            [x_mf[t[0], 0], x_mf[t[0], 1], x_mf[t[0], 2]],
            [x_mf[t[1], 0], x_mf[t[1], 1], x_mf[t[1], 2]],
            [x_mf[t[2], 0], x_mf[t[2], 1], x_mf[t[2], 2]]
        ]
    )
    logp_tri.append(
        (logp_mf[t[0]] + logp_mf[t[1]] + logp_mf[t[2]]) / 3.
    )
    minlogp_tri.append(min(logp_mf[t[0]], logp_mf[t[1]], logp_mf[t[2]]))

z_tri = np.array(z_tri)
x_tri = np.array(x_tri)
logp_tri = np.array(logp_tri)
minlogp_tri = np.array(minlogp_tri)

logp_tri.shape, x_tri.shape, z_tri.shape


## Color map

In [ ]:
np.median(logp_tri), np.mean(logp_tri), np.std(logp_tri), np.min(logp_tri), np.max(logp_tri)

In [ ]:
logp_cutoff = -8.
cmin, cmax = -7., -1.
x_boundary = 3.

def cmap(vals):
    return ps.CMAP(np.clip((vals - cmin) / (cmax - cmin), 0., 1.))


if logp_cutoff is None:
    x_tri_ = x_tri
    colors_tri_ = cmap(logp_tri)
    z_tri_ = z_tri
else:
    x_tri_ = x_tri[minlogp_tri > logp_cutoff]
    colors_tri_ = cmap(logp_tri[minlogp_tri > logp_cutoff])
    z_tri_ = z_tri[minlogp_tri > logp_cutoff]


## 2D plot

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = plt.gca()

coll = PolyCollection(z_tri_, facecolors=colors_tri_, edgecolors=colors_tri_, lw=0.5, rasterized=True)
ax.add_collection(coll)

# ax.plot(z_traj[:, 0], z_traj[:, 1], c="black", lw=0.5)

plt.xlabel("$u_1$")
plt.ylabel("$u_2$")

plt.xlim(-3.1, 3.1)
plt.ylim(-3.1, 3.1)

plt.tight_layout()
plt.savefig("../figures/lorenz_latent_likelihood.pdf")


## 3D plot

In [ ]:
# Strongly based on https://discourse.matplotlib.org/t/trisurf-plots-with-independent-color-data/19033/2

fig = plt.figure(figsize=(6,6))
ax = fig.gca(projection='3d')
    
coll = Poly3DCollection(x_tri_, facecolors=colors_tri_, edgecolors=colors_tri_, lw=0.5)
ax.add_collection(coll)

ax.plot(
    x_traj[:, 0],
    x_traj[:, 1],
    x_traj[:, 2],
    c="black",
    lw=0.5
)

ax.set_xlim3d(-x_boundary, x_boundary)
ax.set_ylim3d(-x_boundary, x_boundary)
ax.set_zlim3d(-x_boundary, x_boundary)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

ax.set_xticklabels([""]*3)
ax.set_yticklabels([""]*3)
ax.set_zticklabels([""]*3)

# ax.view_init(elev=20, azim=-70)  # Good view
ax.view_init(elev=20, azim=-70+180)  # Back view
# ax.view_init(elev=20, azim=60)  # Great side view
# ax.view_init(elev=90, azim=-70)  # Top-down view of x-y plane

plt.tight_layout()
# plt.show()
plt.savefig("../figures/lorenz_likelihood_3d_back.pdf")
